In [29]:
import cv2
import face_recognition

In [30]:
# Load known faces and their names
known_faces_encodings = []
known_faces_names = []

In [31]:
# Function to add known faces and their encodings from image files
def add_known_face_from_image(image_path, name):
    # Load the image
    image = face_recognition.load_image_file(image_path)

    # Find all face encodings in the image
    face_encodings = face_recognition.face_encodings(image)

    if len(face_encodings) == 0:
        print(f"No face found in the image: {image_path}")
        return

    # Use only the first face found in the image
    face_encoding = face_encodings[0]

    # Add the face encoding and the corresponding name to the database
    known_faces_encodings.append(face_encoding)
    known_faces_names.append(name)
    print(f"Face encoding added for {name}.")

In [38]:
# Add known faces to the database from image files
add_known_face_from_image("1Rohit_Tensai.jpg", "Tensai Sir")
add_known_face_from_image("BillGates.jpg", "Billi Uncle")
add_known_face_from_image("elon.jpg", "Elon bhaiya")
add_known_face_from_image("mark_zuckerberg.jpg", "Mark broo")
add_known_face_from_image("asish.jpg", "Ashish")
add_known_face_from_image("sukutta.jpg", "subhankar")

Face encoding added for Tensai Sir.
Face encoding added for Billi Uncle.
Face encoding added for Elon bhaiya.
Face encoding added for Mark broo.
Face encoding added for Ashish.
Face encoding added for Mulla Maghiya.


In [37]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

# Capture video from the webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Resize frame to speed up face recognition
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (OpenCV) to RGB color (face_recognition)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Only process every other frame to speed up face recognition
    if process_this_frame:
        # Find all face locations and face encodings in the current frame
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # Compare each face in the current frame with known faces in the database
            matches = face_recognition.compare_faces(known_faces_encodings, face_encoding)
            name = "Unknown"

            # Check if any known face matches
            if True in matches:
                first_match_index = matches.index(True)
                name = known_faces_names[first_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since we scaled down the frame
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with the name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.8, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Security Check', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
video_capture.release()
cv2.destroyAllWindows()